In [11]:
import json
import numpy as np
from activation_functions import Tanh, Sigmoid
from mse import mse, mse_derivative
from mlp import Dense, train, predict
from utils import get_config_params, get_data, get_training_amount
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [12]:
with open('./config.json', 'r') as f:
    config = json.load(f)

ej, learning_rate, training_percentage, max_epochs, bias, beta, epsilon, optimizer = get_config_params(config)

input_data, expected_data = get_data(2)
training_amount = get_training_amount(len(input_data), training_percentage)

flattened_input = []
for i in range(len(input_data)):
    flattened_input.append([item for sublist in input_data[i] for item in sublist])

training_set = flattened_input[:training_amount]
training_expected = expected_data[:training_amount]
testing_set =  np.reshape(flattened_input[training_amount:], (len(input_data)-training_amount, 35, 1))
testing_expected = expected_data[training_amount:]
X = np.reshape(training_set, (training_amount, 35, 1))
Y = np.reshape(training_expected, (training_amount, 1, 1))
# print(X)
print(Y)

[[[0]]

 [[1]]

 [[0]]

 [[1]]

 [[0]]]


In [13]:
network = [
        Dense(35, 32, optimizer_type=optimizer),
        Sigmoid(),
        Dense(32, 2,optimizer_type=optimizer),
        Sigmoid()
    ]
network


In [14]:
# train
train(network, mse, mse_derivative, X, Y, epochs=10000)

1/10000, error=0.34854171949894686
2/10000, error=0.3444567025737918
3/10000, error=0.3408488004468516
4/10000, error=0.337250467530199
5/10000, error=0.33361878066832085
6/10000, error=0.3299374845813599
7/10000, error=0.3261997134818998
8/10000, error=0.32240464575506605
9/10000, error=0.31855614173443964
10/10000, error=0.3146617014335083
11/10000, error=0.31073151014363554
12/10000, error=0.30677758831360835
13/10000, error=0.302813075550143
14/10000, error=0.2988516602994918
15/10000, error=0.29490715305536763
16/10000, error=0.290993190347092
17/10000, error=0.28712304681483014
18/10000, error=0.2833095243005119
19/10000, error=0.27956488235906657
20/10000, error=0.27590077574336597
21/10000, error=0.2723281716403745
22/10000, error=0.2688572314909549
23/10000, error=0.265497156549778
24/10000, error=0.2622560097696214
25/10000, error=0.2591405362239464
26/10000, error=0.2561560082374261
27/10000, error=0.2533061192692434
28/10000, error=0.2505929434542514
29/10000, error=0.24801

In [19]:
points = []
for i in range(len(testing_set)):
    z = predict(network, testing_set[i])
    points.append([testing_set[i], testing_expected[i], z[i%2,0]])
for point in points:
    print(f"Input:" +
        #    {point[0]} +
             f"Expected:{point[1]} Result:{point[2]}")

Input:Expected:1 Result:0.035405706536931265
Input:Expected:0 Result:0.4795208548386617
Input:Expected:1 Result:0.0003824786278363904
Input:Expected:0 Result:0.998930413993039
Input:Expected:1 Result:0.5590273936846809
